# Facenet embeddings

- [repo](https://github.com/davidsandberg/facenet/blob/master/src/validate_on_lfw.py)

##### Imports

In [1]:
import importlib
import os,sys
import tensorflow as tf
from tqdm import tqdm
import time
import numpy as np

%matplotlib inline

In [2]:
project_dir = os.path.realpath('..')

if not project_dir in sys.path: sys.path.append(project_dir)

In [3]:
facenet = importlib.import_module('src.facenet.src.facenet', project_dir)
from src.utils import load_hdf

### Image files

In [4]:
FILE = '../data/interim/embeddings/juul_day_mtcnn.hdf5'

image_paths = load_hdf(FILE, keys=['image_paths'], print_results=True)['image_paths']

image_paths = list(map(lambda x: x.decode('utf-8'), image_paths))

Dataset image_paths loaded with 385407 lines


### Graph

In [5]:
batch_size = 512
steps,remainder = divmod(len(image_paths), batch_size)
steps += 1 if remainder > 0 else 0
print(steps, remainder)

753 383


In [6]:
config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
g = tf.Graph()
inference_time = []
with g.as_default():
    with tf.Session(config=config) as sess:
        facenet.load_model('../models/20180402-114759/20180402-114759.pb')
        embeddings_tensor = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        embeddings = np.zeros((len(image_paths), embeddings_tensor.get_shape()[1]))
        images_tensor = tf.get_default_graph().get_tensor_by_name("input:0")
        phase_train = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        for step in tqdm(range(steps)):
            start = step * batch_size
            end = min(len(image_paths), (step + 1) * batch_size)
            input_data = facenet.load_data(image_paths[start:end], False, False, 160)
            time_start = time.time()
            embeddings[start:end, :] = sess.run(embeddings_tensor, 
                                                feed_dict={images_tensor: input_data, 
                                                                phase_train: False})
            batch_duration = time.time() - time_start
            inference_time.append(batch_duration)

W0719 12:17:05.202857 140234912388928 deprecation.py:323] From /home/neuro/RG/juul/process/src/facenet/src/facenet.py:373: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.
W0719 12:17:05.203795 140234912388928 deprecation_wrapper.py:119] From /home/neuro/RG/juul/process/src/facenet/src/facenet.py:374: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.



Model filename: ../models/20180402-114759/20180402-114759.pb


100%|██████████| 753/753 [36:48<00:00,  2.87s/it]


In [7]:
print('shapes', embeddings.shape)
print('mean', embeddings.mean(), '+-', embeddings.std())

shapes (385407, 512)
mean 0.0012723121992448847 +- 0.04417585531629799


In [8]:
np.save('../data/interim/embeddings/juul_day_facenet_mtcnn.npy', embeddings)